In [1]:
import zipfile, io, json, os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd

In [2]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
if 'basketball.zip' not in files and 'basketball.sqlite' not in files:
        api = KaggleApi()
        #config_dir = os.getcwd()
        #config_file = 'kaggle.json'
        #api.config = os.path.join(config_dir, config_file)
        api.authenticate()
        api.dataset_download_files('wyattowalsh/basketball') #downloads to cwd
elif 'basketball.zip' in files and 'basketball.sqlite' not in files:
    with zipfile.ZipFile('basketball.zip', 'r') as zip_ref:
        zip_ref.extractall()
    os.remove('daily_execution_pipeline.yml')
    os.remove('monthly_execution_pipeline.yml')

if 'basketball.sqlite' in files:
    %load_ext sql
    %sql sqlite:///basketball.sqlite

In [27]:
%%sql
nba_home_stats << 
SELECT TEAM_ID_HOME, GAME_DATE, TEAM_ABBREVIATION_HOME,
AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612746

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable nba_home_stats


In [28]:
%%sql
nba_away_stats <<
SELECT TEAM_ID_AWAY, GAME_DATE, TEAM_ABBREVIATION_AWAY, 
AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg
FROM Game
WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612746

 * sqlite:///basketball.sqlite
Done.
Returning data to local variable nba_away_stats


In [26]:
%%sql
WITH home_games AS (
    SELECT TEAM_ID_HOME, GAME_DATE, TEAM_ABBREVIATION_HOME,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_HOME_avg,
        AVG(AST_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_HOME_avg,
        AVG(STL_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_HOME_avg,
        AVG(TOV_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_HOME_avg,
        AVG(PF_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_HOME_avg,
        AVG(PTS_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_HOME_avg
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612746
),
WITH away_games AS (
    SELECT TEAM_ID_AWAY, GAME_DATE, TEAM_ABBREVIATION_AWAY, 
        AVG(FGM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_AWAY_avg,
        AVG(FGA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FG3M_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_AWAY_avg,
        AVG(FTM_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_AWAY_avg,
        AVG(FTA_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_AWAY_avg,
        AVG(OREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_AWAY_avg,
        AVG(DREB_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS DREB_AWAY_avg,
        AVG(AST_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS AST_AWAY_avg,
        AVG(STL_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS STL_AWAY_avg,
        AVG(TOV_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS TOV_AWAY_avg,
        AVG(PF_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PF_AWAY_avg,
        AVG(PTS_AWAY) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS PTS_AWAY_avg
    FROM Game
    WHERE Season >= 2019 AND TEAM_ID_HOME = 1610612746
)
SELECT * FROM home_games, away_games

 * sqlite:///basketball.sqlite
(sqlite3.OperationalError) near "away_games": syntax error
[SQL: WITH home_games AS (
    SELECT TEAM_ID_HOME, GAME_DATE, TEAM_ABBREVIATION_HOME,
        AVG(FGM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGM_HOME_avg,
        AVG(FGA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FGA_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FG3M_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FG3M_HOME_avg,
        AVG(FTM_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTM_HOME_avg,
        AVG(FTA_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS FTA_HOME_avg,
        AVG(OREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PRECEDING AND 1 PRECEDING) AS OREB_HOME_avg,
        AVG(DREB_HOME) OVER (ORDER BY GAME_DATE ROWS BETWEEN 15 PREC